# My Model 1a (49 variable)

In [19]:
from keras_tuner import HyperModel
import keras_tuner as kt

import numpy as np
import pandas as pd
#!pip install keras_tuner
#import keras_tuner

import keras
from keras import models, layers, metrics, Sequential
from keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf
from keras.regularizers import l1,l2,l1_l2
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate
from tensorflow.keras.models import Model

import tensorflow as tf
import random

import keras_tuner
from keras_tuner import HyperModel
import keras_tuner as kt

from sklearn.preprocessing import MinMaxScaler

In [20]:
char_core = [
          'acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
          'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
          'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
          'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
          'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']

In [21]:
def import_firm(q=0.2, scaled = False):
    df_firm = pd.read_pickle('firm_df49.pkl')
    df_firm.index = pd.to_datetime(df_firm.index).to_period('M')
    # chars_data 
    chars_data = df_firm.copy()
    factors = chars_data.groupby('jdate').apply(
        lambda x: x[char_core].apply(lambda z: x[z >= z.quantile(1 - q)]['predicted_return'].mean() -
                    x[z <= z.quantile(q)]['predicted_return'].mean()))
    if scaled == True:
        scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
        factors_scaled = scaler.fit_transform(factors)
        factors_scaled_df = pd.DataFrame(factors_scaled, columns=factors.columns)
        factors_scaled_df.index = factors.index
        merged_latent = pd.merge(df_firm, factors_scaled_df, left_on='jdate', right_on=factors_scaled_df.index, how='inner')
    elif scaled == False:
        merged_latent = pd.merge(df_firm, factors, left_on='jdate', right_on=factors.index, how='inner')
    merged_latent.index = df_firm.index
    return merged_latent

merged_latent = import_firm(q=0.2, scaled = False)

# merged data 1
firm_col = merged_latent.columns.tolist()[18:18+49]
latent_col = merged_latent.columns.tolist()[18+49+1:]
print(len(firm_col), len(latent_col))

KeyboardInterrupt: 

In [3]:
#merged_latent.to_pickle('merged_latent49.pkl')
merged_latent = pd.read_pickle("merged_latent49.pkl")

# merged data 1
firm_col = merged_latent.columns.tolist()[18:18+49]
latent_col = merged_latent.columns.tolist()[18+49+1:]
print(len(firm_col), len(latent_col))

49 49


In [22]:
df_train = merged_latent[merged_latent.index < pd.Period((str(1995)+"-1"),freq='M')]
df_val = merged_latent[(merged_latent.index >= pd.Period((str(1995)+"-1"),freq='M')) & (merged_latent.index < pd.Period((str(2000)+"-1"),freq='M'))]
df_test = merged_latent[merged_latent.index >= pd.Period((str(2000)+"-1"),freq='M')]

merged_train_firm = df_train[firm_col]
merged_train_latent = df_train[latent_col]
merged_train_y = df_train['predicted_return']
print(merged_train_firm.shape, merged_train_latent.shape, merged_train_y.shape)

merged_val_firm = df_val[firm_col]
merged_val_latent = df_val[latent_col]
merged_val_y = df_val['predicted_return']
print(merged_val_firm.shape, merged_val_latent.shape, merged_val_y.shape)

merged_test_firm = df_test[firm_col]
merged_test_latent = df_test[latent_col]
merged_test_y = df_test['predicted_return']
print(merged_test_firm.shape, merged_test_latent.shape, merged_test_y.shape)

(88347, 49) (88347, 49) (88347,)
(31851, 49) (31851, 49) (31851,)
(90007, 49) (90007, 49) (90007,)


----

In [23]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

def multi_input_model():
    dropout =0.
    l1_reg = 0.0
    l2_reg =0.0
    # Define the first input with 134 features
    # input 1 is the 49 observed
    # input 2 is the 49 latent variable
    input1 = Input(shape=(49,))

    # Input 2
    input2 = Input(shape=(49,))
    x1 = Dense(4, activation='relu')(input2)
    x1 = Dense(49, activation='relu')(x1)

    # Multiply the outputs of the first branch and the second input element-wise
    merged = Multiply()([input1, x1])
    # Define output layer
    # Create the second branch of the network
    x2 = Dense(32, activation='relu')(merged)
    x2 = Dense(16, activation='relu')(x2)
    #x2 = Dense(8, activation='relu')(x2)
    output = Dense(1, activation='linear')(x2)


    # Create the model with two inputs and one output
    model = Model(inputs=[input1, input2], outputs=output)
    # optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001, beta_1=0.92) # best
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[my_metric_fn])
    return model 

In [24]:
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()
# 10 Trials for Prediction
merged_train_latent_all = np.concatenate((merged_train_latent, merged_val_latent))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    model = multi_input_model()
    model.fit([merged_train_firm, merged_train_latent], merged_train_y,
              validation_data=([merged_val_firm, merged_val_latent], merged_val_y),
              epochs=30,
              batch_size = 128,
              callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)]
              )
    print(model.evaluate([merged_test_firm, merged_test_latent], merged_test_y))
    y_hat = model.predict([merged_test_firm, merged_test_latent]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

Epoch 1/30
691/691 [==============================] - 5s 5ms/step - loss: 0.0121 - my_metric_fn: 0.0056 - val_loss: 0.0146 - val_my_metric_fn: 0.0058
Epoch 2/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0120 - my_metric_fn: 0.0117 - val_loss: 0.0146 - val_my_metric_fn: 0.0061
Epoch 3/30
691/691 [==============================] - 4s 6ms/step - loss: 0.0119 - my_metric_fn: 0.0212 - val_loss: 0.0145 - val_my_metric_fn: 0.0095
Epoch 4/30
691/691 [==============================] - 4s 5ms/step - loss: 0.0116 - my_metric_fn: 0.0446 - val_loss: 0.0141 - val_my_metric_fn: 0.0282
Epoch 5/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0112 - my_metric_fn: 0.0828 - val_loss: 0.0139 - val_my_metric_fn: 0.0378
Epoch 6/30
691/691 [==============================] - 3s 4ms/step - loss: 0.0108 - my_metric_fn: 0.1075 - val_loss: 0.0139 - val_my_metric_fn: 0.0282
Epoch 7/30
2813/2813 [==============================] - 9s 3ms/step - loss: 0.0141 - my_metric_fn: 0

2813/2813 [==============================] - 9s 3ms/step - loss: 0.0139 - my_metric_fn: 0.0411
[0.013860148377716541, 0.041073109954595566]
2813/2813 [==============================] - 8s 3ms/step

Epoch 1/30
691/691 [==============================] - 5s 5ms/step - loss: 0.0121 - my_metric_fn: 0.0072 - val_loss: 0.0145 - val_my_metric_fn: 0.0094
Epoch 2/30
691/691 [==============================] - 3s 5ms/step - loss: 0.0118 - my_metric_fn: 0.0287 - val_loss: 0.0141 - val_my_metric_fn: 0.0290
Epoch 3/30
691/691 [==============================] - 3s 5ms/step - loss: 0.0114 - my_metric_fn: 0.0625 - val_loss: 0.0138 - val_my_metric_fn: 0.0452
Epoch 4/30
691/691 [==============================] - 4s 5ms/step - loss: 0.0110 - my_metric_fn: 0.0909 - val_loss: 0.0137 - val_my_metric_fn: 0.0482
Epoch 5/30
2813/2813 [==============================] - 9s 3ms/step - loss: 0.0138 - my_metric_fn: 0.0547
[0.013815618120133877, 0.054731857031583786]
2813/2813 [==============================] - 7s 3ms

In [25]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.15328355096760926


In [9]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.15328355096760926


---

In [15]:
def Autoencoder(lr=0.0001):
    input_img = Input(shape=(49, ))
    encoded1 = Dense(4, activation='sigmoid')(input_img)
    decoded1 = Dense(49, activation='sigmoid')(encoded1)
    
    autoencoder = Model(inputs=input_img, outputs=decoded1)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    autoencoder.compile(loss='mean_squared_error', optimizer=opt, metrics=[my_metric_fn])
    return autoencoder

autoencoder = Autoencoder()

In [16]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

def multi_input_model():
    dropout =0.2
    l1_reg = 0.01
    l2_reg =0.01
    # Define the first input with 134 features
    # input 1 is the 49 observed
    # input 2 is the 49 latent variable
    input1 = Input(shape=(49,))

    # Input 2
    input2 = Input(shape=(49,))
    x1 = Dense(4, activation='relu')(input2)
    x1 = Dense(49, activation='relu')(x1)

    # Multiply the outputs of the first branch and the second input element-wise
    merged = Multiply()([input1, x1])
    # Define output layer
    # Create the second branch of the network
    x2 = Dense(32, activation='relu')(merged)
    x2 = Dense(16, activation='relu')(x2)
    #x2 = Dense(8, activation='relu')(x2)
    output = Dense(1, activation='linear')(x2)


    # Create the model with two inputs and one output
    model = Model(inputs=[input1, input2], outputs=output)
    # optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001, beta_1=0.92) # best
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[my_metric_fn])
    return model 

In [17]:
autoencoder = Autoencoder()
model = multi_input_model()

print(model.layers[1].get_weights()[0].shape)
print(model.layers[3].get_weights()[0].shape)
print(autoencoder.layers[1].get_weights()[0].shape)
print(autoencoder.layers[2].get_weights()[0].shape)

(49, 4)
(4, 49)
(49, 4)
(4, 49)


In [18]:
import random 

seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    
    autoencoder = Autoencoder()
    autoencoder.fit(merged_train_latent, merged_train_latent,
                epochs = 20, batch_size = 216,
                validation_data = (merged_val_latent, merged_val_latent),
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
 
    model = multi_input_model()
    model.layers[1].set_weights(autoencoder.layers[1].get_weights())
    #model.layers[3].set_weights(autoencoder.layers[2].get_weights())
    model.layers[1].trainable = False
   # model.layers[3].trainable = False
    model.fit([merged_train_latent, merged_train_firm], merged_train_y,
                   epochs=50,
                   validation_data=([merged_val_latent, merged_val_firm], merged_val_y),
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(model.evaluate([merged_test_latent, merged_test_firm], merged_test_y))
    y_hat = model.predict([merged_test_latent, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

Epoch 1/20
410/410 [==============================] - 2s 3ms/step - loss: 0.2424 - my_metric_fn: -380.2728 - val_loss: 0.2277 - val_my_metric_fn: -488.7396
Epoch 2/20
410/410 [==============================] - 1s 3ms/step - loss: 0.2138 - my_metric_fn: -333.8434 - val_loss: 0.2001 - val_my_metric_fn: -429.1965
Epoch 3/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1869 - my_metric_fn: -292.0875 - val_loss: 0.1742 - val_my_metric_fn: -373.2765
Epoch 4/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1618 - my_metric_fn: -253.3098 - val_loss: 0.1504 - val_my_metric_fn: -321.9242
Epoch 5/20
410/410 [==============================] - 1s 2ms/step - loss: 0.1390 - my_metric_fn: -216.5074 - val_loss: 0.1290 - val_my_metric_fn: -275.7303
Epoch 6/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1186 - my_metric_fn: -184.7386 - val_loss: 0.1101 - val_my_metric_fn: -234.8516
Epoch 7/20
410/410 [==============================] - 1s 3ms/ste

410/410 [==============================] - 1s 2ms/step - loss: 0.1584 - my_metric_fn: -246.6981 - val_loss: 0.1472 - val_my_metric_fn: -315.0541
Epoch 5/20
410/410 [==============================] - 1s 2ms/step - loss: 0.1359 - my_metric_fn: -211.7004 - val_loss: 0.1261 - val_my_metric_fn: -269.4794
Epoch 6/20
410/410 [==============================] - 1s 2ms/step - loss: 0.1158 - my_metric_fn: -179.9055 - val_loss: 0.1075 - val_my_metric_fn: -229.2496
Epoch 7/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0982 - my_metric_fn: -152.9869 - val_loss: 0.0913 - val_my_metric_fn: -194.1912
Epoch 8/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0829 - my_metric_fn: -128.6086 - val_loss: 0.0773 - val_my_metric_fn: -163.9477
Epoch 9/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0698 - my_metric_fn: -108.1738 - val_loss: 0.0653 - val_my_metric_fn: -138.0619
Epoch 10/20
410/410 [==============================] - 1s 3ms/step - loss: 

410/410 [==============================] - 1s 2ms/step - loss: 0.0663 - my_metric_fn: -102.9869 - val_loss: 0.0618 - val_my_metric_fn: -130.9259
Epoch 10/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0557 - my_metric_fn: -86.2642 - val_loss: 0.0522 - val_my_metric_fn: -110.0087
Epoch 11/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0467 - my_metric_fn: -72.1141 - val_loss: 0.0440 - val_my_metric_fn: -92.3151
Epoch 12/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0392 - my_metric_fn: -60.2939 - val_loss: 0.0370 - val_my_metric_fn: -77.3997
Epoch 13/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0329 - my_metric_fn: -50.4823 - val_loss: 0.0312 - val_my_metric_fn: -64.8564
Epoch 14/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0275 - my_metric_fn: -42.0555 - val_loss: 0.0263 - val_my_metric_fn: -54.3246
Epoch 15/20
410/410 [==============================] - 1s 2ms/step - loss: 0.02

410/410 [==============================] - 1s 3ms/step - loss: 0.0261 - my_metric_fn: -39.9598 - val_loss: 0.0250 - val_my_metric_fn: -51.3459
Epoch 16/20
410/410 [==============================] - 1s 3ms/step - loss: 0.0219 - my_metric_fn: -33.3280 - val_loss: 0.0211 - val_my_metric_fn: -42.9951
Epoch 17/20
410/410 [==============================] - 1s 3ms/step - loss: 0.0184 - my_metric_fn: -27.7330 - val_loss: 0.0178 - val_my_metric_fn: -36.0002
Epoch 18/20
410/410 [==============================] - 1s 3ms/step - loss: 0.0155 - my_metric_fn: -23.2589 - val_loss: 0.0151 - val_my_metric_fn: -30.1435
Epoch 19/20
410/410 [==============================] - 1s 3ms/step - loss: 0.0130 - my_metric_fn: -19.3996 - val_loss: 0.0128 - val_my_metric_fn: -25.2402
Epoch 20/20
410/410 [==============================] - 1s 3ms/step - loss: 0.0110 - my_metric_fn: -16.1230 - val_loss: 0.0109 - val_my_metric_fn: -21.1356
Epoch 1/50
2761/2761 [==============================] - 7s 2ms/step - loss: 0.0113

410/410 [==============================] - 1s 2ms/step - loss: 0.0388 - my_metric_fn: -59.7632 - val_loss: 0.0371 - val_my_metric_fn: -77.1125
Epoch 13/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0326 - my_metric_fn: -50.0344 - val_loss: 0.0312 - val_my_metric_fn: -64.6253
Epoch 14/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0273 - my_metric_fn: -41.6709 - val_loss: 0.0264 - val_my_metric_fn: -54.1441
Epoch 15/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0229 - my_metric_fn: -34.7673 - val_loss: 0.0223 - val_my_metric_fn: -45.3540
Epoch 16/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0192 - my_metric_fn: -29.0916 - val_loss: 0.0188 - val_my_metric_fn: -37.9878
Epoch 17/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0161 - my_metric_fn: -24.2111 - val_loss: 0.0159 - val_my_metric_fn: -31.8173
Epoch 18/20
410/410 [==============================] - 1s 2ms/step - loss: 0.0136 

In [14]:

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.14131934688759318


---

In [15]:
pred_model = pd.DataFrame()
pred_model['y_real'] = y_real
pred_model['model1'] = y_predict

In [16]:
pred_model

,y_real,model1
jdate,,
2000-01,-0.054008,0.057829
2000-01,0.129941,0.052986
2000-01,0.028313,0.052253
2000-01,-0.041148,0.042085
2000-01,0.243514,0.053958
...,...,...
2022-05,-0.131135,-0.054715
2022-05,-0.208780,-0.085776
2022-05,-0.111692,-0.083689


In [107]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp, n_inputs=49,):
        l1_reg = hp.Choice("l1_ratio", [0.,])
        l2_reg = hp.Choice("l2_ratio", [0.,])
        dropout = hp.Choice("dropout",[ 0.])
        lr = hp.Choice("learning_rate", [0.001])
        
        # Define the first input with 134 features
        # input 1 is the 49 observed
        
        # Input 1
        input1 = Input(shape=(49,))
        x1 = Dense(6, activation='relu')(input1)
        x1 = Dense(49, activation='relu')(x1)
        
        # Input 2
        input2 = Input(shape=(49,))
        
        # Multiply the outputs of the first branch and the second input element-wise
        
        merged = Multiply()([input2, x1])
        # Define output layer
        # Create the second branch of the network
        x2 = Dense(32, activation='relu',kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(merged)
        x2 = Dense(16, activation='relu',kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(x2)
        x2 = Dense(8, activation='relu',kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(x2)
        output = Dense(1, activation='linear')(x2)

        
        # Create the model 
        model = Model(inputs=[input1, input2], outputs=output)
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [128,256]),
            **kwargs,
        )

In [108]:
max_trials = 200
train_epochs = 200

tuner = keras_tuner.RandomSearch(MyHyperModel(),objective='val_loss',max_trials=max_trials,overwrite=True,)
tuner.search([merged_train_latent, merged_train_firm], merged_train_y, 
             epochs=train_epochs, 
             validation_data=([merged_val_latent, merged_val_firm], merged_val_y),
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Trial 2 Complete [00h 00m 14s]
val_loss: 0.013964047655463219

Best val_loss So Far: 0.013964047655463219
Total elapsed time: 00h 00m 32s
INFO:tensorflow:Oracle triggered exit


In [99]:
# 10 Trials for Prediction
# 10 Trials for Prediction
merged_train_latent_all = np.concatenate((merged_train_latent, merged_val_latent))
merged_train_firm_all = np.concatenate((merged_train_firm, merged_val_firm))
merged_y_all = np.concatenate((merged_train_y, merged_val_y))
best_hps = tuner.get_best_hyperparameters(5)
print(best_hps[0].values)
print(best_hps[1].values)
print(best_hps[2].values)


{'l1_ratio': 0.0, 'l2_ratio': 0.0, 'dropout': 0.0, 'learning_rate': 0.001, 'batch_size': 256}
{'l1_ratio': 0.0, 'l2_ratio': 0.0, 'dropout': 0.1, 'learning_rate': 0.001, 'batch_size': 128}
{'l1_ratio': 0.0, 'l2_ratio': 0.0, 'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 128}


# 103 Variable

In [27]:
reduce_char = ['absacc','acc','aeavol','age','agr','beta','betasq','bm','bm_ia',
                            'cash','cashdebt','cashpr','cfp','cfp_ia','chatoia','chcsho','chempia',
                            'chinv','chmom','chpmia','chtx','cinvest','convind','currat','depr',
                            'divi','divo','dolvol','dy','ear','egr','ep','gma','grcapx','grltnoa',
                            'herf','hire','idiovol','indmom','invest','ipo','lev','lgr','mom12m',
                            'mom1m','mom36m','mom6m','ms','mve','mve_ia','nincr','operprof','orgcap',
                            'pchcapx_ia','pchcurrat','pchdepr','pchgm_pchsale','pchquick','pchsale_pchinvt',
                            'pchsale_pchrect','pchsale_pchxsga','pchsaleinv','pctacc','pricedelay','ps',
                            'quick','rd','rd_mve','rd_sale','roaq','roavol','roeq','roic','rsup','salecash',
                            'saleinv','salerec','securedind','sgr','sgrvol','sin','sp','stdacc','stdcf','sue',
                            'tang','tb','turn']

In [31]:
# Import Dataset
firm_df = pd.read_pickle('firm_df49.pkl')
firm_df.index = pd.to_datetime(firm_df.index).to_period('M')

# macro-economic variables
# There is one features which has 37 missing values but before year 1980
macro_df = pd.read_pickle('macro_df.pkl')
macro_df.index = pd.to_datetime(macro_df.index, format='%m/%d/%Y')
macro_df.rename(columns={"ep": "ep_macro"}, inplace=True)
macro_index = macro_df.index[1:]
macro_new = macro_df.iloc[:-1, :]
macro_new.index = macro_index
macro_new.index = macro_new.index.to_period('M')

# Scale macro-features
from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
#macro_scaled = scaler.fit_transform(macro_new)
#macro_scaled_df = pd.DataFrame(macro_scaled, columns=macro_new.columns)
#macro_scaled_df.index = macro_new.index

# merged data 1
merged_df = pd.merge(firm_df, macro_new, left_on='jdate', right_on='sasdate', how='inner')
merged_df.index = firm_df.index

In [32]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
       'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
       'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
       'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
       'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']

info_list = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
       'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

macro_col = ['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx','INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S','IPFUELS', 'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV',
       'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV','UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD',
       'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP','SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT',
       'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE','HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'AMDMNOx', 'ANDENOx', 'AMDMUOx', 'BUSINVx','ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500','S&P: indust', 'S&P div yield', 'S&P PE ratio', 'FEDFUNDS',
       'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA','COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM',
       'AAAFFM', 'BAAFFM', 'TWEXAFEGSMTHx', 'EXSZUSx', 'EXJPUSx','EXUSUKx', 'EXCAUSx', 'WPSFD49207', 'WPSFD49502', 'WPSID61',
       'WPSID62', 'OILPRICEx', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL','CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD',
       'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5','PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA',
       'CES0600000008', 'CES2000000008', 'CES3000000008', 'UMCSENTx','DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'VIXCLSx', 'dp', 'ep_macro', 'b/m',
       'ntis', 'tbl', 'tms', 'dfy', 'svar']

macro_core =  ['dp', 'ep_macro', 'b/m','ntis', 'tbl', 'tms', 'dfy', 'svar']

def datasplit(df, dataset):
    df_train = df[df.index < pd.Period((str(1995)+"-1"),freq='M')]
    df_val = df[(df.index >= pd.Period((str(1995)+"-1"),freq='M')) & (df.index < pd.Period((str(2000)+"-1"),freq='M'))]
    df_test = df[df.index >= pd.Period((str(2000)+"-1"),freq='M')]
    if dataset == "firm":
        X_train = df_train[char_core]
        X_val = df_val[char_core]
        X_test = df_test[char_core]
    elif dataset == 'macro':
        X_train = df_train[macro_col]
        X_val = df_val[macro_col]
        X_test = df_test[macro_col]
    elif dataset == "firm_macro":
        X_train = df_train[char_core+macro_col]
        X_val = df_val[char_core+macro_col]
        X_test = df_test[char_core+macro_col]
    elif dataset == "firm_macro8":
        X_train = df_train[char_core+macro_core]
        X_val = df_val[char_core+macro_core]
        X_test = df_test[char_core+macro_core] 
    y_train = df_train['predicted_return']
    y_val = df_val['predicted_return']
    y_test = df_test['predicted_return']
    return (X_train, y_train, X_val, y_val, X_test, y_test, )

In [39]:
merged_df.columns.tolist()[17:]
# merged data 1
firm_col = merged_df.columns.tolist()[17:17+49]
latent_col = merged_df.columns.tolist()[17+49+1:]
print(len(firm_col), len(latent_col))

49 134


In [40]:
merged_latent = merged_df.copy()
df_train = merged_latent[merged_latent.index < pd.Period((str(1995)+"-1"),freq='M')]
df_val = merged_latent[(merged_latent.index >= pd.Period((str(1995)+"-1"),freq='M')) & (merged_latent.index < pd.Period((str(2000)+"-1"),freq='M'))]
df_test = merged_latent[merged_latent.index >= pd.Period((str(2000)+"-1"),freq='M')]

merged_train_firm = df_train[firm_col]
merged_train_latent = df_train[latent_col]
merged_train_y = df_train['predicted_return']
print(merged_train_firm.shape, merged_train_latent.shape, merged_train_y.shape)

merged_val_firm = df_val[firm_col]
merged_val_latent = df_val[latent_col]
merged_val_y = df_val['predicted_return']
print(merged_val_firm.shape, merged_val_latent.shape, merged_val_y.shape)

merged_test_firm = df_test[firm_col]
merged_test_latent = df_test[latent_col]
merged_test_y = df_test['predicted_return']
print(merged_test_firm.shape, merged_test_latent.shape, merged_test_y.shape)

(88347, 49) (88347, 134) (88347,)
(31851, 49) (31851, 134) (31851,)
(90007, 49) (90007, 134) (90007,)


In [48]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

def multi_input_model():
    dropout =0.1
    l1_reg = 0.0
    l2_reg =0.0
    # Define the first input with 134 features
    # input 1 is the 49 observed
    # input 2 is the 49 latent variable
    input1 = Input(shape=(49,))

    # Input 2
    input2 = Input(shape=(134,))
    x1 = Dense(49, activation='relu')(input2)
    x1 = Dense(4, activation='relu')(x1)
    x1 = Dense(49, activation='relu')(x1)

    # Multiply the outputs of the first branch and the second input element-wise
    merged = Multiply()([input1, x1])
    # Define output layer
    # Create the second branch of the network
    x2 = Dense(32, activation='relu')(merged)
    x2 = Dense(16, activation='relu')(merged)
    x2 = Dense(8, activation='relu')(x2)
    output = Dense(1, activation='linear')(x2)


    # Create the model with two inputs and one output
    model = Model(inputs=[input1, input2], outputs=output)
    # optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001, beta_1=0.92) # best
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[my_metric_fn])
    return model 

In [55]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.1465731927794246
